In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from app.ingest.classes import Match
from pathlib import Path
from collections import defaultdict
import pandas as pd
import plotly.express as px

In [ ]:
def t20_dirs():
    for p in Path("matches").glob("*"):
        for g in p.glob("*"):
            if "T20" in str(g):
                yield g

In [ ]:
matches = [Match.from_json(p.read_text()) for d in t20_dirs() for p in d.glob("*.json")]
print("loaded", len(matches), "matches")

In [ ]:
class Statter():
    def __init__(self) -> None:
        self.accumulator = 0.0
        self.instance_count = 0

    def add(self, quantity):
        self.accumulator += quantity
        self.instance_count += 1

    @property
    def mean(self):
        return self.accumulator / self.instance_count

In [ ]:
stats = {}

over_tots = defaultdict(Statter)
for m in matches:
    for inns in m.innings:
        for over in inns.overs:
            over_tots[over.over].add(over.runs)
stats = [{"over": k, "division": "all matches", "avg runs": v.mean} for k, v in over_tots.items()]

over_tots = defaultdict(Statter)
for m in matches:
    if m.info.gender == "male":
        for inns in m.innings:
            for over in inns.overs:
                over_tots[over.over].add(over.runs)
stats.extend({"over": k, "division": "males", "avg runs": v.mean} for k, v in over_tots.items())

over_tots = defaultdict(Statter)
for m in matches:
    if m.info.gender == "female":
        for inns in m.innings:
            for over in inns.overs:
                over_tots[over.over].add(over.runs)
stats.extend({"over": k, "division": "females", "avg runs": v.mean} for k, v in over_tots.items())

In [ ]:
df = pd.DataFrame(stats)

In [ ]:
fig = px.scatter(df, x="over", y="avg runs", color="division")
fig.show()

In [ ]:
fig=px.scatter(df,x="over", y="avg runs", facet_col="division")
fig.show()